# Prepare synthetic data 
It is going to be much easier to develop if you have realistic synthetic data.
Here we take a sql query that generates a single tabular output.
We run that query against the live identifiable data once.
We then use the [Synthetic Data Vault](https://sdv.dev/SDV/index.html) to prepare a synthetic model of those data.
The code below serves as a vignette for that process but will need adjusting to match the exact contents of the original query.

More complex examples that include multiple tables with joins and dependencies are also possible.

This notebook should be run interactively just once

## Set-up, query and return the data as datafame
The query lives in `./src/api` where `.` represents the project root.
If you run this JupyterNotebook using the local **Makefile** and `make run` then that query will be automatically copied here.

So first steps should be (from _this_ directory)
```sh
make build
make run
```
then navigate to http://uclvlddpragae07:8091/lab/tree/steve/work/synth_test_data.ipynb

In [ ]:
import os
from pathlib import Path

import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# Construct the PostgreSQL connection
uds_host = os.getenv("UDS_HOST")
uds_user = os.getenv("UDS_USER")
uds_passwd = os.getenv("UDS_PWD")

emapdb_engine = create_engine(
    f"postgresql://{uds_user}:{uds_passwd}@{uds_host}:5432/uds"
)

In [ ]:
# Read the sql file into a query 'q' and the query into a dataframe
q = Path("query.sql").read_text()
df = pd.read_sql_query(q, emapdb_engine)

## Generate a synthetic version of the real data

Use the table above to generate the metadata you need for the synthetic data

https://sdv.dev/SDV/user_guides/relational/relational_metadata.html#relational-metadata

In [ ]:
# Minimal imports
from sdv import SDV, Metadata

In [ ]:
# PostgreSQL returns datetimes with tz info which sdv does not seem to be able to handle
def remove_timezone(df, col_name: str) -> pd.DataFrame:
    """sdv does not like timezones"""
    df[col_name] = df[col_name].dt.tz_localize(None)
    return df

### Prepare data and metadata

In [ ]:
tz_cols = [
    "valid_from",
    "scheduled_datetime",
    "status_change_time",
    "admission_time",
    "discharge_time",
]
for col in tz_cols:
    df = remove_timezone(df, col)

sdv doesn't always recognise the columns correctly. Here we specify data_of_birth explicitly as a date.

In [ ]:
fields = {
    "date_of_birth": {
        "type": "datetime",
        "format": "%Y-%m-%d",
    }
}

Prepare the metadata

In [ ]:
metadata = Metadata()
metadata.add_table(
    name="tabpid",
    data=df,
    fields_metadata=fields,
)

In [ ]:
# Inspect the conversion that metadata.add_table did to the dataframe that you loaded
metadata.get_table_meta("tabpid")

Prepare the table(s)

In [ ]:
tables = dict(tabpid=df)

Fit the model

In [ ]:
sdv = SDV()
sdv.fit(metadata, tables)

Inspect the original data

Inspect the synthetic data

In [ ]:
sdv.sample_all()["tabpid"].head()

### Save the synthetic data

Options
- save the model and not the synthetic data (but then you need *sdv* to run the model)
- save the data (need some care with type conversions if you use csv etc.)

In [ ]:
sdv.save("sdv_model.pkl")

In [ ]:
sample = sdv.sample_all()
sample_df = sample["tabpid"]
sample_df.to_pickle("sample_df.pkl")